In this mini-course, we study bootstrap. Majority of the lecture materials are derived from Dr. Alan Polansky's lecture note in 2014 in Northern Illinois University.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats
from sklearn.utils import resample
from scipy.stats import iqr

%matplotlib inline

In [2]:
# !pip install astropy

### I. Jackknife

We will motivate bootstrap by jacknife from a historical point of view. Let $x=(x_{1}, x_{2},...x_{n})'$ be an i.i.d. random sample from a distribution $F$, where the CDF function itself is unknown. Let $\theta$ be the parameter of $F$. Let $\hat{\theta}=\hat{\theta}_{n}(x)$ be an estimator of the true parameter $\theta$. The sub-index $n$ denotes the total sample size. The quality of the estimator can be based on MSE (bias and standard error): $Bias(\hat{\theta}_{n})=E_{\theta}(\hat{\theta}_{n})-\theta$, $SE(\hat{\theta}_{n})=(Var(\hat{\theta}_{n}))^{0.5}$, and $MSE=E(\theta-\hat{\theta}_{n})^{2}=Var(\hat{\theta}_{n})+(Bias(\hat{\theta}_{n}))^{2}$. If we want to reduce the bias of the estimator when the sample size is not large, it's not always easy. For example, if we want to estimate the mean while reducing the bias, it's doable, because we can just do $\hat{\theta}_{n}=\frac{1}{n-1}\sum_{i=1}^{n}(x_{i}-\bar{x}_{n})$. The problem gets a bit harder, if we want to estimate the median. Luckily, Laplace (1973) solved this problem. One can show that the following is true under some regularility conditions: $\sqrt{n}(\hat{\theta}_{n}-\theta) \xrightarrow{d} N(0, \frac{1}{4(f(\theta))^{2}})$ where $\theta$ denotes the population median. However, notice that to get the variance, we will need to do some type of kernel density estimation, which could be challenging given the sample size and the choice of bandwidth. Moreover, once the problem goes beyond nice functions such as mean and median, the problem get even tougher. 

The earliest solution to this bias-reduction problem is to apply the jackknife estimator. Quenouille (1949) solved this problem by the following result:

##### Proposition:
Let $x=(x_{1}, x_{2},...x_{n})'$ be an i.i.d. random sample from a distribution $F$, where the CDF function itself is unknown. Let $\theta$ be the parameter of $F$. Let $\hat{\theta}=\hat{\theta}_{n}(x)$ be an estimator of the true parameter $\theta$. Assume that $E(\hat{\theta}_{n})$ takes on an explicit form of $E(\hat{\theta}_{n})=\theta+\frac{1}{n}a_{1}(F)+\frac{1}{n^{2}}a_{2}(F)+\epsilon(n)$, where as $n \rightarrow \infty$, we have $\epsilon(n) \rightarrow 0$ and $n^{2}\epsilon(n) \rightarrow 0$. Here the functions $a_{1}, a_{2}$ are just some functions dependent on the CDF. Define $\overline{\hat{\theta}_{n, (-i)}}=\frac{1}{n}\sum_{i=1}^{n}\hat{\theta}_{n, (-i)}$ to be the average of all the leave-one-out (LOO) estimators. Then $E((n-1)(\overline{\hat{\theta}_{n, (-i)}}-\hat{\theta}_{n}))=\frac{1}{n}a_{1}(F)+\frac{2n-1}{n^{2}(n-1)}a_{2}(F)+\widetilde{\epsilon}(n-1)$. Since the term $\frac{1}{n}a_{1}(F)$ is the first term of the bias, this suggests that we can use $(n-1)(\overline{\hat{\theta}_{n, (-i)}}-\hat{\theta}_{n})$ to estimate the bias of $\hat{\theta}_{n}$ (for bias reduction). So the **bias-corrected jackknife estimator of the parameter** $\theta$ is given by $\hat{\theta}_{jackknife}=\hat{\theta}_{n}-(n-1)(\overline{\hat{\theta}_{n, (-i)}}-\hat{\theta}_{n})$ so that it's guaranteed that the new estimator has a lower bias than the original estimator so that $Bias(\hat{\theta}_{jackknife})<Bias(\hat{\theta}_{n})$.

The idea of the jackknife estimator is to use resampling to correct the bias by the LOO procedure. The bias of the new jackknife estimator will be smaller than the original bias of $\hat{\theta}_{n}$. Of course, we can keep reducing the bias by doing this again and again but the tradeoff will be the bloated variance. The term $\widehat{Bias}_{jacknife}(\hat{\theta})=(n-1)(\overline{\hat{\theta}_{n, (-i)}}-\hat{\theta}_{n})$ is called the **Quenouille's estimator of bias**, or simply the **jacknife estimator of bias**. This is the precursor of bootstrap. It turns out that the jackknife algorithm is a linear approximation of the bootstrap.

We have talked about the jacknife estimator of bias. So let's push forward by exploring the jacknife estimator of the standard error. Tukey (1958) made the observation that we can rewrite the jackknife estimator as $\hat{\theta}_{jackknife}=\frac{1}{n}\sum_{i=1}^{n}\tilde{\theta}_{n,(-i)}$ where $\tilde{\theta}_{n,(-i)}=n\hat{\theta}_{n}-(n-1)\hat{\theta}_{n,(-i)}$ are called **Tukey's pseudo-values**. Tukey observed that the pseudo-values are approximately i.i.d. and $Var(\tilde{\theta}_{n,(-i)}) \approx Var(\sqrt{n}\hat{\theta}_{n,(-i)})$. This suggests that we can use $\frac{1}{n}Var(\hat{\theta}_{n,(-i)})$ to estimate $Var(\hat{\theta}_{n})$. 

##### Proposition:
The **jackknife estimator of the variance** is given by $\widehat{Var}_{jacknife}(\hat{\theta})=\frac{1}{n(n-1)}\sum_{i=1}^{n}(\tilde{\theta}_{n,(-i)}-\frac{1}{n}\sum_{i=1}^{n}\tilde{\theta}_{n,(-i)})^{2}$ where $\tilde{\theta}_{n,(-i)}=n\hat{\theta}_{n}-(n-1)\hat{\theta}_{n,(-i)}$ are pseudo-values. The **jackknife estimator of the standard error** is therefore $\widehat{SE}_{jacknife}(\hat{\theta})=\sqrt{\widehat{Var}_{jacknife}(\hat{\theta})}$

One can also show the following result easily by tedious computation (see Efron and Tibshirani 1993):

##### Lemma:
$\widehat{Var}_{jacknife}(\hat{\theta})=\frac{1}{n(n-1)}\sum_{i=1}^{n}(\tilde{\theta}_{n,(-i)}-\frac{1}{n}\sum_{i=1}^{n}\tilde{\theta}_{n,(-i)})^{2}=\frac{n-1}{n}\sum_{i=1}^{n}(\hat{\theta}_{n,(-i)}-\overline{\hat{\theta}_{n,(-i)}})^{2}$

Jackknife works well when 'smoothness' is consumed. It works well when the CDF behaves nice (continuous differentiable). It works well when $\hat{\theta}_{n}$ is a smooth function of the data (say mean, variance, skewness, correlation etc.). It does not work well for weird objects such as quantiles, median and robust estimators. This is synthesized by the Miller's theorem below, which can be proved by Taylor's expansion:

##### Miller's Theorem:

Let $x=(x_{1}, x_{2},...x_{n})'$ be an i.i.d. random sample from a distribution $F$, where the CDF function itself is unknown. Let $\theta$ be the parameter of $F$. Let $\hat{\theta}=\hat{\theta}_{n}(x)$ be an estimator of the true parameter $\theta$. Let $\mu, \sigma^{2}$ denote the population mean and variance of $F$. Assume finite variance as well as $g(\mu)=\theta$ for some function $g(.)$ so that $g(\bar{x}_{n})=\hat{\theta}_{n}$. If the function $g(.)$ is continously differentiable at a neighborhood of the population mean and $g'(\mu) \neq 0$. Then $\widehat{SE}_{jacknife}(\hat{\theta}_{n}) \xrightarrow{p} \widehat{SE}(\hat{\theta}_{n})$.

Let's use an example here. Suppose we have an i.i.d. sample size of 7 and want to estimate the parameters 1) $\theta=\mu$, and 2) $\theta=log(\mu)$. Let's obtain the jackknife estimator of bias, variance and the bias-corrected jackknife estimator itself:

In [3]:
data = np.array([0.82,0.77,0.74,0.75,0.74,0.73,0.66])
n=len(data)
resamples = jackknife_resampling(data)
print(resamples.shape)
print(resamples)

test_statistic = np.mean

(7, 6)
[[0.77 0.74 0.75 0.74 0.73 0.66]
 [0.82 0.74 0.75 0.74 0.73 0.66]
 [0.82 0.77 0.75 0.74 0.73 0.66]
 [0.82 0.77 0.74 0.74 0.73 0.66]
 [0.82 0.77 0.74 0.75 0.73 0.66]
 [0.82 0.77 0.74 0.75 0.74 0.66]
 [0.82 0.77 0.74 0.75 0.74 0.73]]


Currently, the "astropy" package only supports jackknife for simple functions such as mean or standard errors etc. So the first question is easy to solve. For the second question, we need to program it from the bottom:

In [4]:
test_statistic = np.mean
estimate, bias, stderr, conf_interval = jackknife_stats(data, test_statistic, 0.95)
print("The bias-corrected jackknife estimate:", estimate)
print("'jackknife estimator of bias (Quenouille's estimator of bias):", bias)
print("jackknife estimator of standard error:", stderr)

The bias-corrected jackknife estimate: 0.7442857142857151
'jackknife estimator of bias (Quenouille's estimator of bias): -6.661338147750939e-16
jackknife estimator of standard error: 0.01810776508745875


We see that the jackknife estimate of bias is 0 essentially, which makes sense since we are doing the mean. For the second problem, here is our strategy: 1) compute $\bar{x}_{n}$ and $\hat{\theta}_{n}=log(\bar{x}_{n})$; 2) compute $\hat{\theta}_{n,(-i)}=log(\bar{x}_{n,(-i)})$ for each $i=1,2,...n$ and then compute $\overline{\hat{\theta}_{n, (-i)}}=\frac{1}{n}\sum_{i=1}^{n}\hat{\theta}_{n, (-i)}$; 3) compute the bias-corrected jackknife estimate by using the jackknife estimate of the bias $\widehat{Bias}_{jacknife}(\hat{\theta})=(n-1)(\overline{\hat{\theta}_{n, (-i)}}-\hat{\theta}_{n})$; 4) compute the jackknife estimate of the standard error $\widehat{Var}_{jacknife}(\hat{\theta})=\frac{n-1}{n}\sum_{i=1}^{n}(\hat{\theta}_{n,(-i)}-\overline{\hat{\theta}_{n,(-i)}})^{2}$ using the lemma above.

In [5]:
x=data # an array object
n=len(x)
x_n_bar=np.mean(x)
theta_hat_n=np.log(x_n_bar) # user-defined function
r1=resamples.mean(axis=1) # this is x_n_bar_LOO
r2=np.log(r1) # this is a vector of theta_hat_n_LOO
theta_hat_n_LOO_mean=r2.mean()
r3=r2-theta_hat_n_LOO_mean

## output
jackknife_bias_estimator=(n-1)*(theta_hat_n_LOO_mean-theta_hat_n)
jackknife_se_estimator=np.sqrt(((n-1)/n)*(np.power(r3,2).sum()))
bias_corrected_jackknife_estimator=theta_hat_n-jackknife_bias_estimator

print(jackknife_bias_estimator)
print(jackknife_se_estimator)
print(bias_corrected_jackknife_estimator)

-0.00029551316762399527
0.024302335494069523
-0.2950347801224136


Let's create a jackknife function that can utilize user-defined functions:

In [96]:
def Quenouille_Jackknife(data, func):
    """
    'Data' must be an numpy array object (otherwise there will be a warning message).
    'Function' should be a univariate function that takes the sample mean of the original data 
     as its argument (it could be a user-defined smooth function).
    The function calculates 3 objects and placed them in a dictionary:
    1) jackknife estimator of bias, 
    2) jackknife estimator of standard error,  
    3) bias-corrected jackknife estimator
    
    Jackknife works the best for smooth functions by Miller's Theorem.
    """
    if isinstance(data, np.ndarray) is False:
        print('Warning: Data need to be an array type, Stop!')
    else:
        n=len(data)
        x_n_bar=np.mean(data)
        theta_hat_n=func(x_n_bar) # user-defined function
        
        r1=resamples.mean(axis=1) # this is x_n_bar_LOO
        r2=func(r1) # this is a vector of theta_hat_n_LOO
        theta_hat_n_LOO_mean=r2.mean()
        r3=r2-theta_hat_n_LOO_mean
        
        ## output
        jackknife_bias_estimator=(n-1)*(theta_hat_n_LOO_mean-theta_hat_n)
        jackknife_se_estimator=np.sqrt(((n-1)/n)*(np.power(r3,2).sum()))
        bias_corrected_jackknife_estimator=theta_hat_n-jackknife_bias_estimator
        
        return {"jackknife bias estimator": jackknife_bias_estimator,
                "jackknife standard error estimator": jackknife_se_estimator, 
                "bias-corrected jackknife estimator": bias_corrected_jackknife_estimator}

In [97]:
output1=Quenouille_Jackknife(data=data, func=np.log)
print(output1)

def identity(x):
    return x

output2=Quenouille_Jackknife(data=data, func=identity)
print(output2)

{'jackknife bias estimator': -0.00029551316762399527, 'jackknife standard error estimator': 0.024302335494069523, 'bias-corrected jackknife estimator': -0.2950347801224136}
{'jackknife bias estimator': -6.661338147750939e-16, 'jackknife standard error estimator': 0.01810776508745875, 'bias-corrected jackknife estimator': 0.7442857142857151}


### II. The Basic Idea of Bootstrap

We now delve into bootstrap. Boostrap extends the idea of LOO and utilizes something called an empirical distribution function (EDF). Usually the parameters we estimate are called **functional parameters**. Simply put, a parameter $\theta \in \Theta$ is called a functional parameter if it can be determined from the population CDF $F(.)$, that is, $\theta=T(F)$ for some function T. In applications, most of the objects we see in classical statistics courses such as mean, quantiles ($q_{\alpha}=inf\{t|F(t) \ge \alpha\}$) are all functional parameters. 

We now need to study empirical distribution functions:

##### Definition:
Let $\mathcal{F}$ be a collection of distributions denoted by $F$. $\mathcal{F}$ is called a **parametric** family iff $\forall F \in \mathcal{F}$, $F$ can be represented by $F_{\theta}$ such that $\theta \in \Theta = \{\theta|\theta=T(F), F \in \mathcal{F}\} \subseteq \mathbb{R}^{p}$ if $p$ is finite. It is a **nonparametric family** if $p=\infty$.

##### Definition:
We define the **empirical distribution function** (EDF) as $\bar{F}_{n}(x)=\frac{1}{n}\sum_{i=1}^{n}I(x \ge x_{i})$, which is the number of $x_{i}$'s that is less than $x$ normalized by ${\frac{1}{n}}$ (proportion of  $x_{i} \le x$).

##### Definition:
If $Q_{n}(\theta)$ is a random function with parameter $\theta \in \Theta$, then we say that $Q_{n}(\theta)$ **converges uniformly in probability** to $Q_{0}(\theta)$ iff $sup_{\theta \in \Theta}|Q_{n}(\theta)-Q_{0}(\theta)| \xrightarrow{p} 0$.

The Use of EDF is essential for nonparametric bootstrap. The following theorem provides the theoretic foundation:

##### Thereom:
Let $x=(x_{1}, x_{2},...x_{n})'$ be an i.i.d. random sample from a distribution $F$ (univariate), where the CDF function itself is unknown. Let $\theta$ be the parameter of $F$. Let $\bar{F}_{n}(x)=\frac{1}{n}\sum_{i=1}^{n}I(x \ge x_{i})$ be the EDF. Then for $\forall x$:
   1. $E(\bar{F}_{n}(x))=F(x)$ and $Var(\bar{F}_{n}(x))=\frac{1}{n}F(x)(1-F(x))$.
   2. Pointwise consistency: $\bar{F}_{n}(x) \xrightarrow{p} F(x)$.
   3. Uniform convergence in probability: $Pr(limsup_{n \rightarrow \infty}(\bar{F}_{n}(x)-F(x))=0)=1$ (this implies pointwise consistency, but not vice versa).
   
Item 3) is also called the **Glivenko-Cantelli Lemma**, which is weaker form of the **Glivenko-Cantelli Theorem**, the latter of which ensures the uniform convergence of the EDF to CDF almost surely (almost everywhere in Lebsgue's integration theory).

The bootstrap problem asks the following question. Suppose we want to understand the sampling distribution of $\hat{\theta}_{n}=T(\hat{F})$ for an i.i.d sample. Recall that $\hat{\theta}_{n}$ is a random variable so let's denote its CDF and pdf by $H_{n}(t|F)=Pr(\hat{\theta}_{n} \le t |x \sim F)$ and $h_{n}(t|F) =H_{n}'(t)$. The question is whether we can estimate the distribution of $\hat{\theta}_{n}$ which explicitly depends on $F$ but $F$ is unknown. Luckily, we can try the 'plug-in' estimator by using the EDF to replace $F$. Specifically, suppose we have a linear functional defined as $\int g(x)dF(x)$. Under some regularity conditions, we can show that $\int g(x)d\bar{F}(x)=\frac{1}{n}\sum_{i=1}^{n}g(x_{i})$. Using this fact, we can see that many parameters can be estimated this way. As the first example, suppose $\theta=T(F)=\int xdF(x)$ is the mean functional. The plug-in estimator becomes $\hat{\theta}_{n}=T(\hat{F})=\int xd\bar{F}_{n}(x)=\frac{1}{n}\sum_{i=1}^{n}x_{i}$. Similaly, the population moment can be estimated by sample moment. As another example, let $\theta=T(F)=inf\{x \in \mathbb{R}| F(x) \ge 1\}$. Writing down the sample analog gives us $\hat{\theta}_{n}=T(\hat{F})=inf\{x \in \mathbb{R}| \bar{F}(x) \ge 1\}=x_{[n]}$, which is the _nth_ largest order statistic. 

Since we have the theoretic backbone of EDF convergence, as well as the capacity to replace the CDF with the EDF easily, we can start laying out more details of the bootstrap. The idea of bootstrap is to create a separete world, called the 'bootstrap world' based on the sample, and then use the relationship between the bootstrap world and the sample to mimic the relationship between the sample and the population. Specifically, the EDF allows us to generate pseudo-samples. This is essentially the sample of the sample, and our hope is that this sample can help us say something about the distribution of the statistic, which is used to estimate the parameter in the population. In particular, we can draw the bootstrap sample $x_{1}^{*},x_{2}^{*},...x_{n}^{*}$ from the EDF with replacement, each time generating a pseudo sample in the 'bootstrap world'. By using this relationship, We can name the **bootstrap estimate** of $H_{n}(t|F)=Pr(\hat{\theta}_{n} \le t |x \sim F)$ to be $\hat{H}_{n}(t|\hat{F})$, and our hope is that $\hat{H}_{n}(t|\hat{F})=Pr^{*}(\hat{\theta}_{n}^{*} \le t |x_{1}^{*},x_{2}^{*},...x_{n}^{*} \sim \hat{F}) \approx H_{n}(t|F)$. This is certainly from an asymptotic perspective. 

To implement bootstrap, we can invoke the Monte Carlo simulation (notice that bootstrap itself does not require simulation if we can do an exhaustive enueration, but Monte Carlo simulation makes it fast). Simulating a sample $x_{1}^{*},x_{2}^{*},...x_{n}^{*}$ conditional on $x$ is equivalent to taking a random sample of size $n$ with replacement from the original sample. So here is the basic bootstrap algorithm:

##### Algorithm (Basic Bootstrap):
   1. Let $b=1$. Simulate a sample from the EDF, compute $\hat{\theta}_{n,b}^{*}$ on the pseudo sample $x_{b}^{*}=(x_{1,b}^{*},x_{2,b}^{*},...x_{n,b}^{*})$. Each sample is taken by using sampling with replacement with size $n$.
   2. Iterate step 1 until $b=1,2,...B$ is large enough to invoke asymptotic theories (Usually $B \ge 1000$). 

Let's do a coarse exercise related to confidence interval involving bootstrap. Suppose we have a few data points in a 'DataFrame' object. We are interested in the population mean $\mu$ as the parameter of interest. Let's use $B=1000$ as the bootstrap sample size. Let's calculate the 95% confidence interval for the population mean. Denote $\delta=\bar{x}_{n}-\mu$. If we knew this distribution we could find $\delta_{0.025}, \delta_{0.975}$, which are the 0.025 and 0.975 critical values of $\delta$. Then we’d have $Pr(\delta_{0.025}\le \bar{x}_{n}-\mu \le \delta_{0.975})=0.95$, which gives the 95% confidence interval of $\mu \in [\bar{x}_{n}-\delta_{0.975}, \bar{x}_{n}-\delta_{0.025}]$. 

The bootstrap principle offers a practical approach to estimating the distribution of $\delta=\bar{x}_{n}-\mu$. It says that we can approximate $\delta=\bar{x}_{n}-\mu$ by $\delta^{*}=\bar{x}_{n}^{*}-\bar{x}$. Since $\delta^{*}$ can be simulated by computer through resampling the original data, and by the bootstrap principle, $Pr(\delta_{0.025} \le \bar{x}_{n}-\mu \le \delta_{0.975})=Pr(\delta_{0.025} \le \bar{x}_{n}^{*}-\bar{x}_{n} \le \delta_{0.975})$. Luckily, we can obtain the critical values from the EDF by $\delta^{*}=\bar{x}_{n}^{*}-\bar{x}_{n}$ and approximate the cutoff values by the empirical cutoff values. Thus our confidence interval becomes $\mu \in [\bar{x}_{n}-\delta_{0.975}^{*}, \bar{x}_{n}-\delta_{0.025}^{*}]$.

In [8]:
datavec = {'Price': [11.9,22.3,34,6.27,14.56,6.76,45.08,9.12,56.34,67.43,0.95,9.4,57,33.35,78,96.45]}
df = pd.DataFrame(datavec, columns = ['Price'])

B=5000 # bootstrap iteration 5000 times
sample_mean=df.mean()[0]
print('sample mean: ', sample_mean)
print('bootstrap size: ', B, '\n')
df_boot_list = []
for i in range(B):
    df_boot_pseudo=resample(df, replace=True).reset_index(drop=True) # do not set the seed
    df_boot_list.append(df_boot_pseudo)
df_boot = pd.concat(df_boot_list, axis=1) # obtaining the bootstrap sample, with each column as one bootstrap sample

deltas1=df_boot.apply(np.mean) # taking the mean of each sample
#print(deltas1.head())
deltas2=df_boot.apply(np.mean)-sample_mean # subtract everything by the original sample_mean
#print(deltas2.head())
deltas2.sort_values(inplace=True)
quantiles=deltas2.quantile([0.025, 0.975]).to_dict()
print('CI for the population mean:\n', "[", sample_mean-quantiles[0.975], sample_mean-quantiles[0.025], "]")

sample mean:  34.306875
bootstrap size:  5000 

CI for the population mean:
 [ 19.87104687499999 47.452749999999995 ]


### III. Asymptotic Theories of Bootstrap

We know that bootstrap relies on asymptotic theories. The idea of bootstrap is to create a pseudo sample by sampling with replacement. Instead of using the asymptotic distribution directly to approximate $H_{n}(t|F)=Pr(\hat{\theta}_{n} \le t |x \sim F)$, we use $\hat{H}_{n}(t|\hat{F})=Pr^{*}(\hat{\theta}_{n}^{*} \le t |x_{1}^{*},x_{2}^{*},...x_{n}^{*} \sim \hat{F}) \approx H_{n}(t|F)$. We say that that the estimator of the CDF for the test statistic is consistent iff  $\hat{H}_{n}(t|\hat{F}) \xrightarrow{p} H_{n}(t|F)$. Next theorem is the "Miller's theorem" for the bootstrap version:

##### Thereom:
Let $x=(x_{1}, x_{2},...x_{n})'$ be an i.i.d. random sample from a distribution $F$, where the CDF function itself is unknown. Let $\theta$ be the functional parameter of $F$ so that $\theta=T(F)$. Let $\hat{\theta}=\hat{\theta}_{n}(x)$ be an estimator of the true parameter $\theta$. Let $\mu$ denote the population mean. Assume $g(\mu)=\theta$ for some function $g(.)$ so that $g(\bar{x}_{n})=\hat{\theta}_{n}$. If the function $g(.)$ is continously differentiable at a neighborhood of the population mean and $g'(\mu) \neq 0$. The bootstrap estimator $\hat{H}_{n}(t|\hat{F})$ is consistent.

The next question is how good is the bootstrap estimate? To understand this, we need to understand the concepts of asymptotic refinement as well as asymptotically pivotal statistics:

##### Definition:
A test statistic is **asymptotically pivotal** if its asymptotic distribution does not depend on any nuisance parameters.

##### Definition:
The bootstrap is said to provide an **asymptotic refinement** if the approximation by bootstrap is of higher order than the approximation achieved by the standard asymptotic approach.

Based on the definitions above, we can see that many conventional statistics are asymptotically pivotal. Yet many are not as well. In general, statistics that resemble the $\sqrt{n}-$consistent, studentized statistic are usually asymptotically pivotal. Examples including $\frac{\sqrt{n}(\bar{x}_{n}-\mu)}{s_{n}^{2}}$, the Wald statistic, Lagrange multiplier test statistic, and likelihood ratio tes statistics are all asymptotically pivotal statistics. 

To quantify the degree of asymptotic statistic, we need to understand the notion of 'higher order' by introducing stochastic order notation as well as the edgeworth expansion. 

##### Definition:
Let $\{a_{n}\}$ and $\{b_{n}\}$ be a sequence of deterministic real numbers. We say that the sequence is **asymtotically bounded** iff $\exists M>0, N \in \mathbb{Z}^{+}$ such that $|\frac{a_{n}}{b_{n}}| \le M $ for $\forall n \ge N$, or $x_{n}=O(y_{n})$.

##### Definition:
Let $\{x_{n}\}$ be a sequence of random variables and let $\{a_{n}\}$ be a sequence of deterministic real numbers. We say that $\{\frac{x_{n}}{a_{n}}\}$ is **stochastically bounded** iff $\forall \epsilon>0$, $\exists M>0, N>0$ such that $Pr(|\frac{x_{n}}{a_{n}}|>M)<\epsilon$ for $\forall n>N$, or $x_{n}=O_{p}(a_{n})$.

Consider the standard central limit theorem (CLT) for the i.i.d. cases. In many asymptotic theories, we often write $\sqrt{n}(\hat{\theta}_{n}-\theta) \xrightarrow{d} N(0,\sigma^{2})$.  This means that $Pr(\frac{1}{\sigma}\sqrt{n}(\hat{\theta}_{n}-\theta) \le t)=\Phi(t)+R_{1}$. Since CLT is based on a truncated power-series expansion, it can be shown that $R_{1}=O(\frac{1}{\sqrt{n}})$. Edgeworth expansion provides asymptotic refinement for CLT so that $Pr(\frac{1}{\sigma}\sqrt{n}(\hat{\theta}_{n}-\theta) \le t)=\Phi(t)+\frac{a_{1}(t)}{\sqrt{n}}\phi(t)+R_{2}$ with $R_{2}=O(\frac{1}{n})$ and $a_{1}(t)$ is a polynomial whose coefficients depend on the moments of $F$. To be more specific, the **Edgeworth series** are series that approximate a probability distribution in terms of its **cumulants**. Edgewroth series is similar to Taylor series in the sense that you can keep the refining happen by adding more expansion terms. That is, we can even further refine the asymptotic expansion by $Pr(\frac{1}{\sigma}\sqrt{n}(\hat{\theta}_{n}-\theta) \le t)=\Phi(t)+\frac{a_{1}(t)}{\sqrt{n}}\phi(t)+\frac{a_{2}(t)}{n}+R_{3}$ with $R_{3}=O(n^{-\frac{3}{2}})$. Notice that as we expand the series, the approximation becomes better and better as $R_{1}>R_{2}>R_{3}$. The problem is that it's extremely cumbersome to explicitly compute these objects, since it involves the characteristic functions, the cumulants as well as Fourier transforms. It turns out that bootstrap can achive edgeworth expansion numerically. However, in order for bootstrap to provide asymptotic refinement, the test statistic must be asymptotically pivotal. 

As an example, consider an i.i.d. random sample $x=(x_{1}, x_{2},...x_{n})'$ from a distribution $F$. Let $\theta=E(x)$ denote the population mean. Assume that $F$ is well behaved in some sense (regularity conditions). Define an asymptotically pivotal quantity such as $\hat{\theta}_{n}=\frac{\sqrt{n}(\bar{x}_{n}-\theta)}{s}$ and let $H_{n}(t|F)=Pr(\hat{\theta}_{n} \le t |x \sim F)$, with $s^{2}=\frac{1}{n-1}\sum_{i=1}^{n}(x_{i}-\bar{x}_{n})^{2}$. Then the bootstrap estimate can achieve asymptotic refinement without explicitly solving the edgeworth expansion formula in the sense that $\hat{H}_{n}(t|\hat{F})=Pr^{*}(\hat{\theta}_{n}^{*} \le t |x_{1}^{*},x_{2}^{*},...x_{n}^{*} \sim \hat{F})=$ $\Phi(t)+\frac{1}{\sqrt{n}}\phi(z)a(t)+R$, where $R$ is a remainder term such that $R=O(\frac{1}{n})$ and $\Phi(.), \phi(.)$ are the respective CDF and pdf of the standard normal distribution, and $a(.)$ is some function. A readable detailed discussion of edgeworth expansion and asymptotic refinement can be found in Cameron and Trivedi (2005). Hall (1992) has a more advanced treatment on this topic.

The bootstrap on an asymptotically pivotal statistic leads to an improved finite-sample performance in the following sense. Let $\alpha$ be the nominal size for a test procedure. Restrict our attentino to asymptotically normal statistics. Usual asymptotic theory produces t-tests with actual size $\alpha+O(n^{-\frac{1}{2}})$ whereas the bootstrap produces t-tests with actual size  $\alpha+O(n^{-1})$. For symmetric two-sided hypothesis tests and confidence intervals the bootstrap on an asymptotically pivotal statistic can be shown to have approximation error  $O(n^{-\frac{3}{2}})$ compared to error  $O(n^{-1})$ using usual asymptotic theory. 

### IV. Bootstrap Bias and Standard Errors

We can use bootstrap to estimate bias and standard errors, just like how we can use jackknife to estimate them. The **bootstrap estimator of bias** $\widehat{Bias}_{boot}(\hat{\theta})=\int tdH_{n}(t|\bar{F}_{n})-\hat{\theta}_{n}=E^{*}(\hat{\theta}_{n}^{*})-\hat{\theta}_{n}$. In practice, we have $\widehat{Bias}_{boot}(\hat{\theta})=\frac{1}{B}\sum_{b=1}^{B}\hat{\theta}_{n,b}^{*}-\hat{\theta}_{n}$, where $b=1,2,...B$, and $\hat{\theta}_{n,b}^{*}$ is each statistic computed on each of the bootstrap resample. The **bootstrap bias-corrected estimator of the parameter** $\theta$ is given by $2\hat{\theta}_{n}-\frac{1}{B}\sum_{b=1}^{B}\hat{\theta}_{n,b}^{*}$.

The **bootstrap estimate of the variance** can be defined similar by writing down the bootstrap analog, and is given by $\widehat{Var}_{boot}(\hat{\theta})=\int (t-\int udH_{n}(u|\bar{F}_{n}))^{2}dH_{n}(t|\bar{F}_{n})$. In practice, $\widehat{Var}_{boot}(\hat{\theta})=\frac{1}{B}\sum_{b=1}^{B}(\hat{\theta}_{n,b}^{*}-\frac{1}{B}\sum_{b=1}^{B}\hat{\theta}_{n,b}^{*})^{2}$. The **bootstrap estimate of the standard error** is thus given by  $\widehat{SE}_{boot}(\hat{\theta})=\sqrt{\widehat{Var}_{boot}(\hat{\theta})}$.

The benefit of these formula is that it's easy to compute. And the framework can be extended to any test statistic. Let's use an example below to estimate the bias and standard error of an estimator (sample median), which is designed to infer about the population median:

In [3]:
datavec = {'Price': [1.45, 6.27, 11.9, 22.88, 65.04]}
df = pd.DataFrame(datavec, columns = ['Price'])

In [4]:
B=5000 # bootstrap iteration 5000 times
seed_num=101
test_statistic_name='sample median'
theta_n_hat=df.median()[0]
print(test_statistic_name, ': ', theta_n_hat)
print('bootstrap size: ', B, '\n')
df_boot_list = []
random.seed(seed_num)
for i in range(B):
    s = random.randint(1,B) # returning a random integer N s.t. a<= N<=b in random.randint(a,b)
    df_boot_pseudo=resample(df, random_state=s, replace=True).reset_index(drop=True) # do not set the seed
    df_boot_list.append(df_boot_pseudo)
df_boot = pd.concat(df_boot_list, axis=1) # obtaining the bootstrap sample, with each column as one bootstrap sample

theta_nb_stars=df_boot.apply(np.median) # taking the median of each sample 
bootstrap_bias_estimate=(1/B)*theta_nb_stars.sum()-theta_n_hat 
print('Bootstrap Bias Estimate: ', bootstrap_bias_estimate)

vec=theta_nb_stars-(1/B)*theta_nb_stars.sum()
vec2=np.power(vec,2)
bootstrap_se_estimate=np.sqrt((1/B)*vec2.sum())
print('Bootstrap SE Estimate: ', bootstrap_se_estimate)

bootstrap_bc_estimator=2*theta_n_hat-(1/B)*theta_nb_stars.sum()
print('Bootstrap Bias-corrected estimator for theta: ', bootstrap_bc_estimator)

sample median :  11.9
bootstrap size:  5000 

Bootstrap Bias Estimate:  3.9023140000000023
Bootstrap SE Estimate:  13.844388436670073
Bootstrap Bias-corrected estimator for theta:  7.997685999999998


Now let's define a function that does the above:

In [5]:
def basic_boot(df, B, random_state, test_statistic_name, test_stat):
    theta_n_hat=df.apply(test_stat)[0]
    print(test_statistic_name, ': ', theta_n_hat)
    print('bootstrap replicate: ', B, '\n')
    
    df_boot_list = []
    random.seed(random_state)
    for i in range(B):
        s = random.randint(1,B) # returning a random integer N s.t. a<= N<=b in random.randint(a,b)
        df_boot_pseudo=resample(df, random_state=s, replace=True).reset_index(drop=True) # do not set the seed
        df_boot_list.append(df_boot_pseudo)
    df_boot = pd.concat(df_boot_list, axis=1) # getting the bootstrap sample, with each column as one bootstrap sample

    theta_nb_stars=df_boot.apply(test_stat) # computing test statistic (theta_n_hat) for each bootstrap sample 
    bootstrap_bias_estimate=(1/B)*theta_nb_stars.sum()-theta_n_hat 
    print('Bootstrap Bias Estimate: ', bootstrap_bias_estimate)

    vec=theta_nb_stars-(1/B)*theta_nb_stars.sum()
    vec2=np.power(vec,2)
    bootstrap_se_estimate=np.sqrt((1/B)*vec2.sum())
    print('Bootstrap SE Estimate: ', bootstrap_se_estimate)

    bootstrap_bc_estimator=2*theta_n_hat-(1/B)*theta_nb_stars.sum()
    print('Bootstrap Bias-corrected estimator for theta: ', bootstrap_bc_estimator)
    
    return {"Bootstrap Estimate of Bias": bootstrap_bias_estimate,
            "Bootstrap Estimate of Standard Error": bootstrap_se_estimate, 
            "Bias-Corrected Bootstrap Estimator:": bootstrap_bc_estimator}

In [6]:
basic_boot(df=df, B=5000, random_state=101, test_statistic_name='sample median', test_stat=np.median)

sample median :  11.9
bootstrap replicate:  5000 

Bootstrap Bias Estimate:  3.9023140000000023
Bootstrap SE Estimate:  13.844388436670073
Bootstrap Bias-corrected estimator for theta:  7.997685999999998


{'Bootstrap Estimate of Bias': 3.9023140000000023,
 'Bootstrap Estimate of Standard Error': 13.844388436670073,
 'Bias-Corrected Bootstrap Estimator:': 7.997685999999998}

Let's try another example using sample interquartile range to estimate population interquartile range. Notice that the standard error is big, because IQR itself is not a smooth function. So the bootstrap estimate of bias and variance is simply not that good. Keep in mind that all these non-smooth functions don't get handled well by bootstrap. A classic textook example for bootstrap failure is the mode. Here let's look at the performance of bootstrap on IQR:

In [106]:
basic_boot(df=df, B=1500, random_state=201, test_statistic_name='sample interquartile range', test_stat=iqr)

sample interquartile range :  16.61
bootstrap size:  1000 

Bootstrap Bias Estimate:  4.831900000000001
Bootstrap SE Estimate:  20.671152294683527
Bootstrap Bias-corrected estimator for theta:  11.778099999999998


{'Bootstrap Estimate of Bias': 4.831900000000001,
 'Bootstrap Estimate of Standard Error': 20.671152294683527,
 'Bias-Corrected Bootstrap Estimator:': 11.778099999999998}

The benefit of being able to estimate the bootstrap standard error is not constricted to getting a better estimate of itself per se. The idea can be extended to even selectin tuning parameters. FOr example, Leger and Romano (1990) used bootstrap to estimate robust statistics involving tuning parameters. During the 80s and 90s, there is a strong emphasis on the concept of robust statistics. To be more specific, suppose we have an i.i.d. sample drawn from a population with parameter of interest $\theta$. Suppose we have a symmetric CDF such that $F(t|\theta)=F(t-\theta)$. Of course, we can use the sample mean to estimate $\theta$, but also, we can use the sample median to estimate $\theta$. The debate right now centers on 'robustness'. And this yields the concept of 'trimmed means'. To be more specific, let $\tau \in [0,\frac{1}{2})$, and let $x_{[1]},x_{[2]}...x_{[n]}$ be the order statistics (ascending order). The trimmed mean is defined to be $\hat{\theta}_{\tau}=\frac{1}{n-2\lfloor n\tau \rfloor}\sum_{i=1+\lfloor n\tau \rfloor}^{n-\lfloor n\tau \rfloor}x_{(i)}$ where $\lfloor.\rfloor$ denotes the largest integer less than or equal to that argument. So if $\tau=0.1$ and $n=100$, we have $\hat{\theta}_{\tau}=\frac{1}{80}\sum_{i=11}^{90}x_{(i)}$. The idea is that the estimator trims off the largetst and smallest $\lfloor n\tau \rfloor$% of the data. So if $\tau$ approaches 0, the estimator approaches to sample mean. And if $\tau$ approaches 0.5, then the estimator approaches sample median. Now the question boils down to what choice of $\tau$ is optimal, or how many data points should be trimmed off in some optimal sense. This is essentially a tuning parameter selection problem. 

If you write down some criterion function that solves this optimization problem. The algebra is really complicated and it could involve theorems related to weak topology. So a natural way to solve this problem is via bootstrap. When we have the symmetry condition on the CDF, the assumption helps us to focus more on the standard error estimate of $\hat{\theta}_{\tau}$. So specifically, we can use bootstrap to estimate the bootstrap standard error for a grid of values of $\tau$ to minimize $\hat{\theta}_{\tau}$. We then estimate the optimal value of $\tau$ using graphical examiniation. This helps us bypass the hard optimization problem. 

#### References:

   - Davison and Hinkley (1997) Bootstrapping and its Applications, Cambridge University Press.
   - Efron, B. (1982) The Jackknife, the Bootstrap and Other Resampling Plans, SIAM.
   - Efron, B. and Tibshirani, R.J. (1993) An Introduction to the Bootstrap, Chapman & Hall.
   - Manly, Bryan F.J. (2007) Randomization, Bootstrap, and Monte Carlo Methods in Biology, Chapman & Hall/CRC Press.
   - Shao, J. and Tu, T. (1995) The Jackknife and Bootstrap, Springer-Verlag.
   - A. Colin Cameron and Pravin K. Trivedi (2005) Microeconometrics: Methods and Applications. Cambridge University Press.
   - Singh, K. (1981), "On the Asymptotic Accuracy of Efron’s Bootstrap", Annals of Statistics, 9,
1187–1195.
   - Hall, P. (1992), The Bootstrap and Edgeworth Expansions, New York, Springer-Verlag.
   - Polansky, A. (2014), STAT673: Lecture Notes on Bootstrap, NIU.
   - Quenouille, Maurice H. (September 1949). "Problems in Plane Sampling". The Annals of Mathematical Statistics. 20(3): 355–375.
   - Tukey, John W. (1958), "Bias and confidence in not quite large samples (abstract)". The Annals of Mathematical Statistics. 29(2): 614. 
   - Léger, C. and Romano, J., (1990), "Bootstrap choice of tuning parameters, Annals of the Institute of Statistical Mathematics", 42, Issue 4, pp.709-735.
   - https://docs.astropy.org/en/stable/api/astropy.stats.jackknife_stats.html